# XGMIX notebook

In [58]:
import numpy as np
import os

In [59]:
%load_ext autoreload
%autoreload 2

from preprocess import data_process,simulate_missing_values
from XGMIX import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Main arguments

In [60]:
# args = {'train_data': "/home/arvindsk/XGMix/expts/xgmix_global_1/train.txt", 
#         "val_data" : "/home/arvindsk/XGMix/expts/xgmix_global_1/val.txt", 
#         "save": None,
#         "window_size": 1000,
#         "smooth_size": 75,
#         "missing":0.0}

In [61]:
args = {'train_data': "/home/arvindsk/XGMix/expts/xgmix_global_22/train.txt", 
        "val_data" : "/home/arvindsk/XGMix/expts/xgmix_global_22/val.txt", 
        "save": "/home/arvindsk/XGMix/expts/xgmix_global_22/",
        "window_size": 8000,
        "smooth_size": 75,
        "missing":0}

## Pre-processing

In [62]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
args = Struct(**args)

win = args.window_size
training, training_labels, validation, validation_labels = data_process(args)
chmlen = training.shape[-1]
sws = args.smooth_size if args.smooth_size%2 else args.smooth_size-1
num_anc = len(np.unique(training_labels))

Loading data...
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_2//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_4//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_train/gen_6//mat_map.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_2//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_4//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_valid/gen_6//mat_vcf_2d.npy
Reading /home/arvindsk/datagen/simulation-output/chm22/xgmix_worldwide_22_v

In [63]:
if args.missing != 0:
    training = simulate_missing_values(training,args.missing)
    validation = simulate_missing_values(validation,args.missing)

In [64]:
try:
    os.mkdir(args.save)
except:
    print("Directory exists")

Directory exists


In [65]:
xgm = XGMIX(chmlen,win,sws,num_anc,args.save,cores=16)

In [66]:
xgm.train(training,training_labels,validation, validation_labels, smoothlite=10000)

Windows done: 0, 
Base Training Accuracy: 0.9844444444444445, Base Validation Accuracy: 0.8233333333333334
Smooth Training Accuracy: 1.0 
Smooth Validation Accuracy: 0.9287179487179488 


In [67]:
# predict using the built-in method.
ypreds = xgm.predict(validation)
print("Accuracy")
print(np.mean(ypreds == validation_labels.numpy()))
print("Confusion matrix")
print(sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel()))

Accuracy
0.9287179487179488
Confusion matrix
[[3288   25   11    5    3    0    1   19]
 [   4   17    1    0    0    0    0    0]
 [   5    2 3917   36   45   28   78    8]
 [   5    1   23  940    5    1   58  114]
 [   9    0   14    9  995    3    5    7]
 [   8    0    7    0    3   97   10    0]
 [   9    0   32   76    1    5 1200   55]
 [  11    4    6   53    1    0   28  412]]


In [68]:
# predict using the standalone function.
ypreds = predict(validation,args.save+"/model.pkl")
print("Accuracy")
print(np.mean(ypreds == validation_labels.numpy()))
print("Confusion matrix")
print(sklearn.metrics.confusion_matrix(ypreds.ravel(), validation_labels.numpy().ravel()))

XGBoostError: [01:15:25] /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:682: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /home/wknd37/anaconda3/lib/libxgboost.so(+0x8871f) [0x7f1d96afb71f]
  [bt] (1) /home/wknd37/anaconda3/lib/libxgboost.so(+0x18895e) [0x7f1d96bfb95e]
  [bt] (2) /home/wknd37/anaconda3/lib/libxgboost.so(XGBoosterUnserializeFromBuffer+0x5e) [0x7f1d96afd9ee]
  [bt] (3) /home/wknd37/anaconda3/lib/python3.7/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x